# NLP 기초 Assignment
## NLP 맛보기 - spam.csv를 가지고 유의미한 해석을 도출해주세요!

In [1]:
# 필요한 모듈 불러오기
import pandas as pd
!pip install glove_python_binary

     |████████████████████████████████| 948 kB 12.9 MB/s 


In [2]:
# 필요한 라이브러리들 올리기
# 사용할 라이브러리들이 import되어있어야 사용할 수 있습니다. 
import gensim
from gensim.models import Word2Vec, FastText
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import re
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Data
- 보시면 아시다시피 spam.csv는 라벨이 있는 데이터입니다. 물론 7주차 주제가 텍스트 기초인만큼 텍스트만 활용하셔도 되고, 라벨까지 활용하셔서 모델을 돌려보셔도 좋습니다 :)

In [13]:
spam = pd.read_csv('/content/drive/MyDrive/NLP/spam.csv')

In [14]:
spam.head(30)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [15]:
spam.iloc[5]['v2']


"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

### 전처리 작업 수행하기

In [16]:
def data_text_cleaning(data):
 
    # 영문자 이외 문자는 공백으로 변환
    only_english = re.sub('[^a-zA-Z]', ' ', data)
 
    # 소문자 변환
    no_capitals = only_english.lower().split()
 
    # 불용어 제거
    stops = set(stopwords.words('english'))
    no_stops = [word for word in no_capitals if not word in stops]
 
    # 어간 추출
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmer_words = [stemmer.stem(word) for word in no_stops]
 
    # 공백으로 구분된 문자열로 결합하여 결과 반환
    return ' '.join(stemmer_words)

In [17]:
spam['v2'] = spam['v2'].apply(data_text_cleaning)

In [18]:
spam.head(30)

,v1,v2
0,ham,go jurong point crazi avail bugi n great world...
1,ham,ok lar joke wif u oni
2,spam,free entri wkli comp win fa cup final tkts st ...
3,ham,u dun say earli hor u c alreadi say
4,ham,nah think goe usf live around though
5,spam,freemsg hey darl week word back like fun still...
6,ham,even brother like speak treat like aid patent
7,ham,per request mell mell oru minnaminungint nurun...
8,spam,winner valu network custom select receivea pri...
9,spam,mobil month u r entitl updat latest colour mob...


## Tokenizing


In [19]:
import nltk

In [20]:
# 예시 코드 코드
from nltk.tokenize import word_tokenize

nltk.download('punkt')
word_tokenize(spam.iloc[5]['v2'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['freemsg',
 'hey',
 'darl',
 'week',
 'word',
 'back',
 'like',
 'fun',
 'still',
 'tb',
 'ok',
 'xxx',
 'std',
 'chgs',
 'send',
 'rcv']

In [21]:
result = [word_tokenize(sentence) for sentence in spam['v2']]

In [22]:
result[3]

['u', 'dun', 'say', 'earli', 'hor', 'u', 'c', 'alreadi', 'say']

## Embedding

- 수업에서 다룬 임베딩 방법에는 One-hot encoding, CBOW, Skip-gram, GloVe, FastText, EMlo가 있었습니다. 다양한 시도와 '비교' 결과를 함께 적어주세요! 파라미터를 조정해가는 과정도 해석에 도움이 될 수 있겠죠 :)

### 1. FastText

In [32]:
from gensim.models import FastText

model = FastText(result, size=100, window=5, min_count=5, workers=4, sg=1)

In [33]:
# 모델을 저장합니다. 
model.save('ft_model')

In [34]:
ft_model = FastText.load('ft_model')

In [31]:
ft_model.similarity('movie', 'good')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.87419605

### 2. Word2Vec


In [86]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

In [93]:
model = Word2Vec(result, min_count=5, size=100, window=2, iter=200, sg=1)

model.save('w2v_model')
w2v_model = Word2Vec.load('w2v_model')

### 2. Glove Embedding

In [88]:
from glove import Corpus, Glove

corpus = Corpus() 

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [89]:
# 모델을 저장합니다. 

glove.save('gv_model')

In [90]:
gv_model = Glove.load('gv_model')
gv_model.most_similar('math')

[('comput', 0.8327851744053579),
 ('valuabl', 0.8264002368395605),
 ('street', 0.8200727623861964),
 ('suit', 0.8185827296030435)]

## 본인이 도출해낸 해석을 적어주세요!

- 유사도, Wordcloud, 이진 분류 모델, Plot 뭐든 상관없으니 분명하고 인상적인 해석을 적어주시면 됩니다.

## 1) 단어 유사도 평가

In [130]:
nouns = [
  "time",
  "year",
  
  "way",
  "day",
  "man",
  "thing",
  "woman",
  "life",
  
  "world",
  "school",

  "student",
  "group",
  
  "problem",
  "hand",
  "part",
  "place",
  "case",
  "week",
  
  "system",
  
  "question",
  "work",

  "number",
  "night",
  "point",
  "home",
  "water",
  "room",
  "mother",
  "area",
  "money",
  
  "fact",
  "month",
  "lot",
  "right",

  "book",
  "eye",
  "job",
  "word",

  "side",
  "kind",
  "head",
  "friend",
  "father",
  "power",
  "hour",
  "game",
  "line",
  "end",
  "member",
  "law",
  "car",
  # "president",
  # "team",
  # "minute",
  # "idea",
  # "kid",
  # "body",
  # "information",
  # "back",
  # "parent",
  # "face",
  # "level",
  # "office",
  # "door",
  # "health",
  # "person",
  # "art",
  # "war",
  # "history",
  # "party",
  # "result",
  # "change",
  # "morning",
  # "reason",
  # "research",
  # "girl",
  # "guy",
  # "moment",
  # "air",
  # "teacher",
  # "force",
  # "education",
]

In [135]:
from nltk.corpus import wordnet # wordnet으로 평가한 similarity와 비슷할 수록 더 좋은 모델로 평가
import random
ft_diff = 0
w2v_diff = 0
for i in range(len(nouns)-1):

  cb = wordnet.synset(nouns[i] + '.n.01')
  ib = wordnet.synset(str(nouns[i +1]) +'.n.01')
  
  wordnet_sim = cb.wup_similarity(ib)
  ft_sim = ft_model.similarity(nouns[i], nouns[i +1])
  w2v_sim = w2v_model.similarity(nouns[i], nouns[i +1])
  ft_diff += abs(wordnet_sim -ft_sim)
  w2v_diff += abs(w2v_sim - wordnet_sim)

print(ft_diff)
print(w2v_diff)



33.87848586311391
7.029612525561276


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if sys.path[0] == '':


In [136]:
# w2v_diff 의  유사도 성능이 훨씬 좋다는 것을 알 수 있다.
